In [1]:
import pandas as pd 
import plotly.graph_objects as go 
import plotly.express as px

In [2]:
# df = pd.read_csv("all_month.csv")
# print(df)

                           time   latitude   longitude    depth   mag magType  \
0      2023-08-02T06:14:09.220Z  35.308167 -118.503833    7.110  1.24      ml   
1      2023-08-02T06:08:54.450Z  33.226167 -115.780000   14.720  0.97      ml   
2      2023-08-02T06:07:45.920Z  33.555167 -116.610833   14.080  0.73      ml   
3      2023-08-02T05:53:53.070Z  35.522500 -118.363333    1.940  0.88      ml   
4      2023-08-02T05:48:55.106Z  58.200500 -155.250400    1.100  0.60      ml   
...                         ...        ...         ...      ...   ...     ...   
11191  2023-07-03T06:37:55.870Z  33.586833 -116.750000   12.190  0.14      ml   
11192  2023-07-03T06:30:15.615Z   4.596700  126.001700  136.122  4.40      mb   
11193  2023-07-03T06:29:26.080Z  51.438100 -177.088900   35.000  2.70      ml   
11194  2023-07-03T06:27:06.360Z  58.185667 -155.302667    1.630  0.11      ml   
11195  2023-07-03T06:26:34.250Z  33.682833 -116.791500   16.930  0.71      ml   

        nst    gap      dmi

In [3]:
def extract_subarea(place):
    return place[0]

In [4]:
def extract_area(place):
    return place[-1]

In [5]:
def extract_date(time):
    return str(time).split(' ')[0]

In [6]:
def extract_weekday(time):
    date = extract_date(time)
    return date + ' - ' + str(time.weekday())

In [7]:
def extract_hour(time):
    t = str(time).split(' ')
    return t[0] + ' - ' + t[1].split(':')[0]    

In [8]:
def fetch_eq_data(period="daily",region="Worldwide",min_mag=1):

    url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/{}.csv'

    if period == 'weekly':
        new_url = url.format('all_week')
    elif period == 'monthly':
        new_url = url.format('all_month')
    else:
        new_url = url.format('all_day')

    df_earthquake = pd.read_csv(new_url)
    df_earthquake = df_earthquake[['time','latitude','longitude','mag','place']]
    place_list = df_earthquake['place'].str.split(', ')
    df_earthquake['sub_area'] = place_list.apply(extract_subarea)
    df_earthquake['area'] = place_list.apply(extract_area)
    df_earthquake = df_earthquake.drop(columns=['place'],axis=1)


    if isinstance(min_mag,int) and min_mag > 0:
        df_earthquake = df_earthquake[df_earthquake['mag'] >= min_mag]
    else:
        df_earthquake = df_earthquake[df_earthquake['mag'] > 0]

    
    df_earthquake['time'] = pd.to_datetime(df_earthquake['time'])
    

    if region in df_earthquake['area'].to_list():
        df_earthquake = df_earthquake[df_earthquake['area']==region]
        max_mag = df_earthquake['mag'].max()
        center_lat = df_earthquake[df_earthquake['mag']== max_mag]['latitude'].values[0]
        center_long = df_earthquake[df_earthquake['mag']==max_mag]['longitude'].values[0]
    else:
        center_lat , center_long = [54,15]

    if period == 'weekly':
        animation_frame_col = 'weekday'
        df_earthquake[animation_frame_col] = df_earthquake['time'].apply(extract_weekday)
    elif period == 'monthly':
        animation_frame_col = 'date'
        df_earthquake[animation_frame_col] = df_earthquake['time'].apply(extract_date)
    else:
        animation_frame_col = 'hours'
        df_earthquake[animation_frame_col] = df_earthquake['time'].apply(extract_hour)
    
    df_earthquake = df_earthquake.sort_values(by='time')
    
    return df_earthquake , center_lat ,center_long


In [9]:
def visualize_eq_data(period = "daily", region = "Worldwide", min_mag = 1):
    df_earthquake , center_lat , center_long = fetch_eq_data(period=period,region=region,min_mag=min_mag)

    if period == 'monthly':
        animation_frame_col = 'date'
    elif period == 'weekly':
        animation_frame_col = 'weekday'
    else:
        animation_frame_col = 'hours'
    
    fig = px.scatter_mapbox(
        data_frame = df_earthquake,
        lat = 'latitude',
        lon = 'longitude',
        center = dict(lat = center_lat,lon = center_long),
        size = 'mag',
        color = 'mag',
        hover_name = 'sub_area',
        zoom =1,
        mapbox_style = 'carto-positron',
        animation_frame = animation_frame_col,
        title = 'Earthquakes'
    )

    fig.show()

    return None 
    

    
    

In [ ]:
visualize_eq_data(period='monthly',region='Worldwide',min_mag=1)